# Lab 1: Set Covering

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$, <br>
determine, if possible, $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$<br>
such that each number between $0$ and $N-1$ appears in at least one list<br>
<br>
$forall \ n \ in \ [0, N-1] \ exists \ i : n \ in \ L_{s_i}$<br>
<br>
and that the total numbers of elements in all $L_{s_i}$ is minimum.

In [1]:
import random
from gx_utils import *
from copy import deepcopy
import numpy as np
import logging
from random import seed, choice
from typing import Callable

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [3]:
logging.basicConfig(format="%(message)s", level=logging.INFO)

In [4]:
class State:
    def __init__(self, data: np.array):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [5]:
def search(
    N,
    initial_state: State,
    goal_test: Callable,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
    beam = 0,
):
    frontier = PriorityQueue()
    state_cost.clear()

    state = initial_state
    state_cost[state] = 0

    while state is not None and not goal_test(state, N):
        count = 0
        for a in possible_actions(state, N):
            count +=1
            if beam and count >= beam:
                break
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    if state:
        logging.info(f"Found a solution for N={N}: w={state_cost[state]:,}; visited {len(state_cost):,} nodes")
    else:
        logging.info("Solution not found!")
    
    return 1 if state else 0

In [6]:
SEED = 42
seed(SEED)

In [7]:
INITIAL_STATE = State(np.array([]))

GOAL = dict()
for N in (5, 10, 20, 100, 500, 1000):
    GOAL[N] = State(np.array(list(set(range(N)))))
    #logging.info(f"Goal:\n{GOAL[N]}")


def goal_test(state, N):
    return state == GOAL[N]

In [8]:
def is_valid(state: list, action: list):
    valid = not all(item in state for item in action)
    return valid

all_lists = dict()
for N in (5, 10, 20, 100, 500, 1000):
    all_lists[N] = problem(N, seed=SEED)

def possible_actions(state: State, N):
    return (m for m in all_lists[N] if is_valid(list(state._data), m))


def result(state: State, action: list):
    return State(np.array(list(set(state.copy_data()).union(action))))

## Breadth-First

In [9]:
for N in (5, 10, 20, 100, 500, 1000):
    if N < 100:
        state_cost = dict()

        final = search(
            N,
            INITIAL_STATE,
            goal_test=goal_test,
            state_cost=state_cost,
            priority_function=lambda s: len(s._data),
            unit_cost=lambda a: len(a),
        )
    else:
        logging.info(f"Breadth-First was too slow to find a solution for N={N}")

Found a solution for N=5: w=5; visited 32 nodes
Found a solution for N=10: w=10; visited 743 nodes
Found a solution for N=20: w=23; visited 15,935 nodes
Breadth-First was too slow to find a solution for N=100
Breadth-First was too slow to find a solution for N=500
Breadth-First was too slow to find a solution for N=1000


## Depth-First

In [10]:
for N in (5, 10, 20, 100, 500, 1000):
    state_cost = dict()

    final = search(
        N,
        INITIAL_STATE,
        goal_test=goal_test,
        state_cost=state_cost,
        priority_function=lambda s: -len(s._data),
        unit_cost=lambda a: len(a),
    )

Found a solution for N=5: w=5; visited 17 nodes
Found a solution for N=10: w=11; visited 62 nodes
Found a solution for N=20: w=28; visited 74 nodes
Found a solution for N=100: w=173; visited 1,603 nodes
Found a solution for N=500: w=1,304; visited 10,778 nodes
Found a solution for N=1000: w=2,893; visited 24,238 nodes


## A*

In [11]:
for N in (5, 10, 20, 100, 500, 1000):
    def h(state):
        if N < 100:
            return int(N / (N + 1 - len(state)))
        return N - len(state)
        

    state_cost = dict()

    final = search(
        N,
        INITIAL_STATE,
        goal_test=goal_test,
        state_cost=state_cost,
        priority_function=lambda s: len(s._data) + h(s._data),
        unit_cost=lambda a: len(a),
    )

Found a solution for N=5: w=5; visited 32 nodes
Found a solution for N=10: w=10; visited 743 nodes
Found a solution for N=20: w=23; visited 15,935 nodes
Found a solution for N=100: w=214; visited 2,163 nodes
Found a solution for N=500: w=1,489; visited 14,368 nodes
Found a solution for N=1000: w=3,717; visited 255,965 nodes
